In [1]:
library(tidyverse)
library(lfe)
library(data.table)
library(dplyr)
library(stargazer)
library(readxl)
library(tidyr)
library(MASS)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transp

In [2]:
df_gains <- read_excel("../data/GAINS/GHGemis_vs_PM25_conc_GAINS_countries_regions_v2.xlsx", sheet = "Export Worksheet")

In [46]:
df2 <- df_gains %>%
    mutate(
        logpm = log(PM25_ANTHROP),
        loggdp = log(GDP_PPP),
        logpop = log(POP),
        loggdppc = log(GDP_PPP / POP),
        loggdppc2 = loggdppc^2,
        logco2 = log(EMIS_CO2_KT),
        logch4 = log(EMIS_CH4_KT)
    ) %>%
    group_by(REGION_4LETTER) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        loggdp0 = loggdp - first(loggdp),
        logpop0 = logpop - first(logpop),
        loggdppc0 = loggdppc - first(loggdppc),
        loggdppc02 = loggdppc2 - first(loggdppc2),
        logco20 = logco2 - first(logco2),
        logch40 = logch4 - first(logch4),
        year0 = IDYEARS - first(IDYEARS),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2),
        logco20xyear0 = logco20 * year0,
        logch40xyear0 = logch40 * year0
    ) %>%
    ungroup()

# Replace NA values with 0
df2 <- df2 %>%
    mutate(
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )

## PM2.5 ANTHOROPOGENIC

In [51]:
summary(
    model1 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 + 
               loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS), 
               data = subset(df2, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS),      data = subset(df2, IDSCENARIOS == "Decarb")) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.62503 -0.04394  0.00427  0.04546  0.98393 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
logco20       -0.072786   0.047009  -1.548  0.12238    
logch40        0.226900   0.037483   6.053 3.43e-09 ***
logco20xyear0  0.005503   0.001951   2.820  0.00505 ** 
logch40xyear0 -0.005109   0.002809  -1.818  0.06978 .  
loggdppc0      2.228163   0.255290   8.728  < 2e-16 ***
loggdppc02     0.091328   0.011896   7.677 1.41e-13 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1279 on 377 degrees of freedom
Multiple R-squared(full model): 0.9362   Adjusted R-squared: 0.9181 
Multiple R-squared(proj model): 0.3647   Adjusted R-squared: 0.1844 
F-statisti

## OUT-OF-COUNTRY CONTRIBUTION

In [55]:
file_path <- "../data/GAINS/PM25_sourcecontrib_GAINS_4letter.xlsx"
df_export <- read_excel(file_path, sheet = "Export Worksheet")
df_long <- df_export %>%
    pivot_longer(cols = -c(IDSCENARIOS, IDYEARS, RECEPTOR_4LETTER),
                 names_to = "Source",
                 values_to = "PM25_Contribution") %>%
    rename(REGION_4LETTER = RECEPTOR_4LETTER)

df_agg_split <- df_long %>%
    group_by(IDSCENARIOS, IDYEARS, REGION_4LETTER) %>%
    summarise(
        PM25_TOTAL = sum(PM25_Contribution, na.rm = TRUE),
        PM25_OUT   = sum(PM25_Contribution[REGION_4LETTER != Source], na.rm = TRUE),
        PM25_SELF  = sum(PM25_Contribution[REGION_4LETTER == Source], na.rm = TRUE),
        .groups = "drop"
    )

df_agg_export <- df_long %>%
    filter(Source != REGION_4LETTER) %>%  # Exclude self-contributions
    group_by(IDSCENARIOS, IDYEARS, Source) %>%
    summarise(
        PM25_EXPORT = sum(PM25_Contribution, na.rm = TRUE),
        .groups = "drop"
    )

df2_agg <- df_agg_split %>%
    left_join(df_agg_export, by = c("IDSCENARIOS", "IDYEARS", "REGION_4LETTER" = "Source"))

df2 <- df2 %>%
    left_join(df2_agg, by = c("IDSCENARIOS", "IDYEARS", "REGION_4LETTER"))

## SELF CONTRIBUTION

In [56]:
df2_self <- df2 %>%
    mutate(
        logpm = log(PM25_SELF)  # Changed to PM25_SELF

    ) %>%
    group_by(REGION_4LETTER, IDSCENARIOS) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2)
    ) %>%
    ungroup()
df2_self <- df2_self %>%
    mutate(
        logpm = ifelse(is.infinite(logpm) | is.na(logpm), 0, logpm),
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )


In [104]:
nrow(subset(df2_self, IDSCENARIOS == "Decarb"))

[1] 485

In [109]:
    model3_baseline <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 + 
               loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS), 
               data = subset(df2_self, IDSCENARIOS == "Baseline" & IDYEARS > 2020))

In [83]:
summary(
    model3 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 + 
               loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS), 
               data = subset(df2_self, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS),      data = subset(df2_self, IDSCENARIOS == "Decarb")) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.34497 -0.04129  0.00255  0.03088  0.47268 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
logco20       -1.134e-01  3.487e-02  -3.253  0.00124 **
logch40        2.250e-03  2.780e-02   0.081  0.93555   
logco20xyear0  3.541e-03  1.447e-03   2.447  0.01488 * 
logch40xyear0 -2.387e-05  2.084e-03  -0.011  0.99087   
loggdppc0      3.527e-01  1.894e-01   1.863  0.06330 . 
loggdppc02     1.339e-02  8.824e-03   1.517  0.12999   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.09484 on 377 degrees of freedom
Multiple R-squared(full model): 0.9436   Adjusted R-squared: 0.9276 
Multiple R-squared(proj model): 0.05364   Adjusted R-squared: -0.215 
F-s

## EXPORT CONTRIBUTION

In [58]:
df2_export <- df2 %>%
    mutate(
        logpm = log(PM25_EXPORT)  # Changed to PM25_EXPORT

    ) %>%
    group_by(REGION_4LETTER, IDSCENARIOS) %>%
    arrange(IDYEARS) %>%
    mutate(
        logpm0 = logpm - first(logpm),
        laglogpm0 = lag(logpm0),
        lag2logpm0 = lag(logpm0, 2)
    ) %>%
    ungroup()
df2_export <- df2_export %>%
    mutate(
        logpm = ifelse(is.infinite(logpm) | is.na(logpm), 0, logpm),
        laglogpm0 = ifelse(is.na(laglogpm0), 0, laglogpm0),
        lag2logpm0 = ifelse(is.na(lag2logpm0), 0, lag2logpm0)
    )


In [59]:
df2_export <- df2_export %>%
    drop_na(PM25_EXPORT, EMIS_CO2_KT, EMIS_CH4_KT, GDP_PPP) %>% 
    filter(PM25_EXPORT > 0, EMIS_CO2_KT > 0, EMIS_CH4_KT > 0, GDP_PPP > 0)

In [110]:
model4_baseline <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 + 
               loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS), 
               data = subset(df2_export, IDSCENARIOS == "Baseline" & IDYEARS > 2020))

In [60]:
summary(
    model4 <- felm(logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 + 
               loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS), 
               data = subset(df2_export, IDSCENARIOS == "Decarb"))
        )


Call:
   felm(formula = logpm0 ~ logco20 + logch40 + logco20xyear0 + logch40xyear0 +      loggdppc0 + loggdppc02 | year0:factor(REGION_4LETTER) + factor(IDYEARS),      data = subset(df2_export, IDSCENARIOS == "Decarb")) 

Residuals:
      Min        1Q    Median        3Q       Max 
-0.259815 -0.030506 -0.000135  0.032506  0.269320 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)  
logco20       -0.050992   0.028390  -1.796   0.0733 .
logch40       -0.005037   0.022297  -0.226   0.8214  
logco20xyear0  0.002715   0.001150   2.361   0.0188 *
logch40xyear0  0.000695   0.001762   0.395   0.6934  
loggdppc0      0.339557   0.152315   2.229   0.0264 *
loggdppc02     0.015394   0.007126   2.160   0.0314 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.0752 on 368 degrees of freedom
Multiple R-squared(full model): 0.9479   Adjusted R-squared: 0.933 
Multiple R-squared(proj model): 0.04212   Adjusted R-squared: -0.2312 
F-stat

##  Only do regressions for _SELF and _EXPORT, use full model for these (9 coeffs)

### _SELF is model3  and _EXPORT is model4

In [112]:
#mvrnorm(1000, coef(model3), vcov(model3), empirical=T)
write.csv(as.data.frame(MASS::mvrnorm(1000, coef(model3_baseline), vcov(model3_baseline), empirical = TRUE)), "mvrnorm_SELF_Contribution_baseline.csv", row.names = FALSE)
write.csv(as.data.frame(MASS::mvrnorm(1000, coef(model3), vcov(model3), empirical = TRUE)), "mvrnorm_SELF_Contribution.csv", row.names = FALSE)

In [62]:
write.csv(getfe(model3)[, c(1, 4, 5)], "fixedeffects_SELF_Contribution.csv", row.names=F)

Warning message in is.estimable(ef, obj$fe):
“non-estimable function, largest error 0.3 in coordinate 5 ("factor(IDYEARS).2050")”
Warning message in getfe.kaczmarz(obj, se, ef = ef, bN = bN, robust = robust, cluster = cluster, :
“Supplied function seems non-estimable”


In [111]:
#mvrnorm(1000, coef(model4), vcov(model4), empirical=T)
write.csv(as.data.frame(MASS::mvrnorm(1000, coef(model4_baseline), vcov(model4_baseline), empirical = TRUE)), "mvrnorm_EXPORT_Contribution_baseline.csv", row.names = FALSE)
write.csv(as.data.frame(MASS::mvrnorm(1000, coef(model4), vcov(model4), empirical = TRUE)), "mvrnorm_EXPORT_Contribution.csv", row.names = FALSE)

In [64]:
write.csv(getfe(model4)[, c(1, 4, 5)], "fixedeffects_EXPORT_Contribution.csv", row.names=F)

Warning message in is.estimable(ef, obj$fe):
“non-estimable function, largest error 0.3 in coordinate 5 ("factor(IDYEARS).2050")”
Warning message in getfe.kaczmarz(obj, se, ef = ef, bN = bN, robust = robust, cluster = cluster, :
“Supplied function seems non-estimable”


In [74]:
dfout = subset(df_long, IDYEARS == 2020 & PM25_Contribution > 0 & IDSCENARIOS == "Baseline" & REGION_4LETTER != Source)
dfout2 = dfout %>% group_by(Source) %>% mutate(PM25_Contribution=PM25_Contribution / sum(PM25_Contribution))
subset(dfout2, Source == "EAFR")

IDSCENARIOS,IDYEARS,REGION_4LETTER,Source,PM25_Contribution
<chr>,<dbl>,<chr>,<chr>,<dbl>
Baseline,2020,KENY,EAFR,0.48507463
Baseline,2020,MIDE,EAFR,0.01940299
Baseline,2020,RSAF,EAFR,0.01194030
Baseline,2020,TANZ,EAFR,0.27313433
Baseline,2020,WAFR,EAFR,0.21044776


In [75]:
write.csv(dfout2[, 3:5], "export_pattern.csv", row.names=F)

In [122]:
names(df2)

[1] "IDSCENARIOS"          "IDYEARS"              "REGION_4LETTER"      
 [4] "REGION_4LETTER_LABEL" "POP"                  "GDP"                 
 [7] "GDP_PPP"              "EMIS_CO2_KT"          "EMIS_CH4_KT"         
[10] "PM25_ANTHROP"         "PM25_TOTAL.x"         "logpm"               
[13] "loggdp"               "logpop"               "loggdppc"            
[16] "loggdppc2"            "logco2"               "logch4"              
[19] "logpm0"               "loggdp0"              "logpop0"             
[22] "loggdppc0"            "loggdppc02"           "logco20"             
[25] "logch40"              "year0"                "laglogpm0"           
[28] "lag2logpm0"           "logco20xyear0"        "logch40xyear0"       
[31] "PM25_TOTAL.y"         "PM25_OUT.x"           "PM25_SELF.x"         
[34] "PM25_EXPORT.x"        "PM25_TOTAL.x.x"       "PM25_OUT.y"          
[37] "PM25_SELF.y"          "PM25_EXPORT.y"        "PM25_TOTAL.y.y"      
[40] "PM25_OUT"             "PM25_SELF"            "PM25_EXPORT"

In [125]:
write.csv(df2[df2$REGION_4LETTER %in% df2$REGION_4LETTER[df2$IDYEARS == 2030], c('IDSCENARIOS', 'IDYEARS', 'REGION_4LETTER', 'POP', 'GDP_PPP', 'EMIS_CO2_KT', 'EMIS_CH4_KT', 'PM25_SELF', 'PM25_EXPORT')], "baseline.csv", row.names=F)

In [116]:
head(df2)

IDSCENARIOS,IDYEARS,REGION_4LETTER,REGION_4LETTER_LABEL,POP,GDP,GDP_PPP,EMIS_CO2_KT,EMIS_CH4_KT,PM25_ANTHROP,⋯,PM25_SELF.x,PM25_EXPORT.x,PM25_TOTAL.x.x,PM25_OUT.y,PM25_SELF.y,PM25_EXPORT.y,PM25_TOTAL.y.y,PM25_OUT,PM25_SELF,PM25_EXPORT
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Baseline,2020,AFGH,Afghanistan,38054941,9.289786,22.48262,23451.44,712.67,42.63,⋯,41.789,1.828,57.294,15.505,41.789,1.828,57.294,15.505,41.789,1.828
Baseline,2020,ALBA,Albania,2942034,11.301931,24.96461,4688.82,150.49,16.50,⋯,9.297,0.742,17.955,8.658,9.297,0.742,17.955,8.658,9.297,0.742
Baseline,2020,ARGE,Argentina,45510399,242.398694,510.02911,138696.29,5964.82,3.37,⋯,2.908,0.336,5.590,2.682,2.908,0.336,5.590,2.682,2.908,0.336
Baseline,2020,ARME,Armenia,2938679,10.762538,28.98772,6692.76,86.93,10.04,⋯,6.083,0.505,13.670,7.587,6.083,0.505,13.670,7.587,6.083,0.505
Baseline,2020,AUST,Austria,8782210,353.917177,317.53936,60160.62,222.75,9.37,⋯,4.296,3.244,9.714,5.418,4.296,3.244,9.714,5.418,4.296,3.244
Baseline,2020,AUTR,Australia,25398177,986.870189,907.28794,371354.65,4983.79,2.43,⋯,2.358,0.076,3.962,1.604,2.358,0.076,3.962,1.604,2.358,0.076


In [119]:
write.csv(df2[, c('IDSCENARIOS', 'IDYEARS', 'REGION_4LETTER', 'REGION_4LETTER_LABEL', 'PM25_ANTHROP', 'PM25_SELF', 'PM25_EXPORT')], "gainssim.csv", row.names=F)